In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [2]:
get_ipython().run_line_magic('matplotlib', 'inline')

In [3]:
df = pd.read_csv('Leads.csv')

In [4]:
df.head().T

,0,1,2,3,4
Prospect ID,7927b2df-8bba-4d29-b9a2-b6e0beafe620,2a272436-5132-4136-86fa-dcc88c88f482,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,3256f628-e534-4826-9d63-4a8b88782852
Lead Number,660737,660728,660727,660719,660681
Lead Origin,API,API,Landing Page Submission,Landing Page Submission,Landing Page Submission
Lead Source,Olark Chat,Organic Search,Direct Traffic,Direct Traffic,Google
Do Not Email,No,No,No,No,No
Do Not Call,No,No,No,No,No
Converted,0,0,1,0,1
TotalVisits,0.0,5.0,2.0,1.0,2.0
Total Time Spent on Website,0,674,1532,305,1428
Page Views Per Visit,0.0,2.5,2.0,1.0,1.0


In [5]:
df.dtypes

Prospect ID                                       object
Lead Number                                        int64
Lead Origin                                       object
Lead Source                                       object
Do Not Email                                      object
Do Not Call                                       object
Converted                                          int64
TotalVisits                                      float64
Total Time Spent on Website                        int64
Page Views Per Visit                             float64
Last Activity                                     object
Country                                           object
Specialization                                    object
How did you hear about X Education                object
What is your current occupation                   object
What matters most to you in choosing a course     object
Search                                            object
Magazine                       

In [6]:
df.columns = df.columns.str.lower().str.replace(' ', '_')
 
string_columns = list(df.dtypes[df.dtypes == 'object'].index)
 
for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

In [7]:
df.converted = (df.converted == 1).astype(int)

In [8]:
df.converted.head()

0    0
1    0
2    1
3    0
4    1
Name: converted, dtype: int32

In [9]:
(df.converted == 1).head()

0    False
1    False
2     True
3    False
4     True
Name: converted, dtype: bool

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [12]:
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)
 
y_train = df_train.converted.values
y_val = df_val.converted.values
 
del df_train['converted']
del df_val['converted']

In [13]:
df_train_full.isnull().sum()

prospect_id                                         0
lead_number                                         0
lead_origin                                         0
lead_source                                        26
do_not_email                                        0
do_not_call                                         0
converted                                           0
totalvisits                                       108
total_time_spent_on_website                         0
page_views_per_visit                              108
last_activity                                      82
country                                          1971
specialization                                   1136
how_did_you_hear_about_x_education               1748
what_is_your_current_occupation                  2135
what_matters_most_to_you_in_choosing_a_course    2153
search                                              0
magazine                                            0
newspaper_article           

In [14]:
df_train_full.converted.value_counts()

0    4560
1    2832
Name: converted, dtype: int64

In [15]:
global_mean = df_train_full.converted.mean()

In [16]:
round(global_mean, 3)

0.383

In [17]:
categorical=['lead_origin','lead_source','do_not_email','do_not_call'
             ,'last_activity','country','specialization', 'how_did_you_hear_about_x_education',
             'what_is_your_current_occupation','what_matters_most_to_you_in_choosing_a_course','search',
            'magazine','newspaper_article','digital_advertisement', 'through_recommendations','receive_more_updates_about_our_courses',
            'tags', 'lead_quality','update_me_on_supply_chain_content','lead_profile','city',
            'asymmetrique_activity_index','asymmetrique_profile_index', 'i_agree_to_pay_the_amount_through_cheque',
            'a_free_copy_of_mastering_the_interview','last_notable_activity']
numerical=['totalvisits','total_time_spent_on_website','page_views_per_visit','asymmetrique_activity_score','asymmetrique_profile_score']

In [18]:
df_train_full[categorical].nunique()

lead_origin                                       5
lead_source                                      19
do_not_email                                      2
do_not_call                                       2
last_activity                                    16
country                                          36
specialization                                   19
how_did_you_hear_about_x_education               10
what_is_your_current_occupation                   6
what_matters_most_to_you_in_choosing_a_course     2
search                                            2
magazine                                          1
newspaper_article                                 2
digital_advertisement                             2
through_recommendations                           2
receive_more_updates_about_our_courses            1
tags                                             26
lead_quality                                      5
update_me_on_supply_chain_content                 1
lead_profile

In [19]:
df_train_full.what_is_your_current_occupation.value_counts()

unemployed              4503
working_professional     560
student                  167
other                     13
housewife                  8
businessman                6
Name: what_is_your_current_occupation, dtype: int64

In [20]:
occupation_unemployed = df_train_full[df_train_full.what_is_your_current_occupation == 'unemployed'].converted.mean()
occupation_working_professional = df_train_full[df_train_full.what_is_your_current_occupation == 'working_professional'].converted.mean()
occupation_student = df_train_full[df_train_full.what_is_your_current_occupation == 'student'].converted.mean()
occupation_other = df_train_full[df_train_full.what_is_your_current_occupation == 'other'].converted.mean()
occupation_housewife = df_train_full[df_train_full.what_is_your_current_occupation == 'housewife'].converted.mean()
occupation_businessman = df_train_full[df_train_full.what_is_your_current_occupation == 'businessman'].converted.mean()

In [21]:
print('what_is_your_current_occupation == unemployed :', round(occupation_unemployed, 3))
print('what_is_your_current_occupation == working_professional :', round(occupation_working_professional, 3))
print('what_is_your_current_occupation == student :', round(occupation_student, 3))
print('what_is_your_current_occupation == other :', round(occupation_other, 3))
print('what_is_your_current_occupation == housewife :', round(occupation_housewife, 3))
print('what_is_your_current_occupation == businessman :', round(occupation_businessman, 3))

what_is_your_current_occupation == unemployed : 0.429
what_is_your_current_occupation == working_professional : 0.916
what_is_your_current_occupation == student : 0.359
what_is_your_current_occupation == other : 0.615
what_is_your_current_occupation == housewife : 1.0
what_is_your_current_occupation == businessman : 0.5


In [22]:
global_mean = df_train_full.converted.mean()
 
df_group = df_train_full.groupby(by='what_is_your_current_occupation').converted.agg(['mean'])
df_group['diff'] = df_group['mean'] - global_mean
df_group['risk'] = df_group['mean'] / global_mean
 
df_group

,mean,diff,risk
what_is_your_current_occupation,,,
businessman,0.500000,0.116883,1.305085
housewife,1.000000,0.616883,2.610169
other,0.615385,0.232268,1.606258
student,0.359281,-0.023835,0.937785
unemployed,0.429491,0.046375,1.121045
working_professional,0.916071,0.532955,2.391102


In [23]:
from IPython.display import display 
 
for col in categorical:
    df_group = df_train_full.groupby(by=col).converted.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['rate'] = df_group['mean'] / global_mean
    display(df_group)

,mean,diff,rate
lead_origin,,,
api,0.308414,-0.074702,0.805014
landing_page_submission,0.360677,-0.022440,0.941427
lead_add_form,0.919861,0.536744,2.400992
lead_import,0.225000,-0.158117,0.587288
quick_add_form,1.000000,0.616883,2.610169


,mean,diff,rate
lead_source,,,
bing,0.166667,-0.216450,0.435028
blog,0.000000,-0.383117,0.000000
click2call,0.750000,0.366883,1.957627
direct_traffic,0.317624,-0.065493,0.829052
facebook,0.225000,-0.158117,0.587288
google,0.400000,0.016883,1.044068
live_chat,1.000000,0.616883,2.610169
nc_edm,1.000000,0.616883,2.610169
olark_chat,0.251768,-0.131349,0.657157


,mean,diff,rate
do_not_email,,,
no,0.401265,0.018148,1.047368
yes,0.174281,-0.208836,0.454903


,mean,diff,rate
do_not_call,,,
no,0.383033,-0.000083,0.999782
yes,1.000000,0.616883,2.610169


,mean,diff,rate
last_activity,,,
approached_upfront,1.000000,0.616883,2.610169
converted_to_lead,0.126437,-0.256680,0.330021
email_bounced,0.092308,-0.290809,0.240939
email_link_clicked,0.239437,-0.143680,0.624970
email_marked_spam,1.000000,0.616883,2.610169
email_opened,0.363137,-0.019980,0.947848
email_received,1.000000,0.616883,2.610169
form_submitted_on_website,0.255102,-0.128015,0.665860
had_a_phone_conversation,0.727273,0.344156,1.898305


...

,mean,diff,rate
asymmetrique_activity_index,,,
01.high,0.313522,-0.069595,0.818344
02.medium,0.428008,0.044891,1.117173
03.low,0.092715,-0.290402,0.242002


,mean,diff,rate
asymmetrique_profile_index,,,
01.high,0.482525,0.099408,1.259473
02.medium,0.304191,-0.078926,0.793990
03.low,0.416667,0.033550,1.087571


,mean,diff,rate
i_agree_to_pay_the_amount_through_cheque,,,
no,0.383117,0.0,1.0


,mean,diff,rate
a_free_copy_of_mastering_the_interview,,,
no,0.39578,0.012663,1.033053
yes,0.35545,-0.027667,0.927785


,mean,diff,rate
last_notable_activity,,,
approached_upfront,1.000000,0.616883,2.610169
email_bounced,0.160000,-0.223117,0.417627
email_link_clicked,0.222222,-0.160895,0.580038
email_marked_spam,1.000000,0.616883,2.610169
email_opened,0.364362,-0.018755,0.951046
email_received,1.000000,0.616883,2.610169
form_submitted_on_website,0.000000,-0.383117,0.000000
had_a_phone_conversation,0.900000,0.516883,2.349153
modified,0.230430,-0.152687,0.601461


In [24]:
from IPython.display import display 
 
for col in numerical:
    df_group = df_train_full.groupby(by=col).converted.agg(['mean'])
    df_group['diff'] = df_group['mean'] - global_mean
    df_group['rate'] = df_group['mean'] / global_mean
    display(df_group)

,mean,diff,rate
totalvisits,,,
0.0,0.411966,0.028849,1.075301
1.0,0.164596,-0.218521,0.429624
2.0,0.315946,-0.067171,0.824674
3.0,0.375361,-0.007756,0.979756
4.0,0.391800,0.008683,1.022663
5.0,0.402226,0.019109,1.049877
6.0,0.389920,0.006804,1.017758
7.0,0.388000,0.004883,1.012746
8.0,0.471910,0.088793,1.231765


,mean,diff,rate
total_time_spent_on_website,,,
0,0.411497,0.028380,1.074077
1,0.200000,-0.183117,0.522034
2,0.090909,-0.292208,0.237288
3,0.222222,-0.160895,0.580038
4,0.000000,-0.383117,0.000000
...,...,...,...
2207,0.000000,-0.383117,0.000000
2217,1.000000,0.616883,2.610169
2226,0.000000,-0.383117,0.000000


,mean,diff,rate
page_views_per_visit,,,
0.00,0.411966,0.028849,1.075301
1.00,0.284351,-0.098766,0.742205
1.14,0.500000,0.116883,1.305085
1.20,0.250000,-0.133117,0.652542
1.22,0.500000,0.116883,1.305085
...,...,...,...
14.00,0.833333,0.450216,2.175141
15.00,0.333333,-0.049784,0.870056
16.00,0.000000,-0.383117,0.000000


,mean,diff,rate
asymmetrique_activity_score,,,
7.0,0.000000,-0.383117,0.000000
8.0,0.000000,-0.383117,0.000000
9.0,0.000000,-0.383117,0.000000
10.0,0.069767,-0.313349,0.182105
11.0,0.073171,-0.309946,0.190988
12.0,0.115152,-0.267965,0.300565
13.0,0.139384,-0.243733,0.363816
14.0,0.406272,0.023155,1.060439
15.0,0.632094,0.248977,1.649872


,mean,diff,rate
asymmetrique_profile_score,,,
11.0,0.500000,0.116883,1.305085
12.0,0.388889,0.005772,1.015066
13.0,0.229730,-0.153387,0.599634
14.0,0.275862,-0.107255,0.720047
15.0,0.281382,-0.101735,0.734455
16.0,0.405010,0.021894,1.057146
17.0,0.467091,0.083974,1.219187
18.0,0.367442,-0.015675,0.959086
19.0,0.739583,0.356466,1.930438


In [25]:
df_train_full[numerical].corrwith(df_train_full.converted)

totalvisits                    0.033651
total_time_spent_on_website    0.363470
page_views_per_visit           0.001593
asymmetrique_activity_score    0.165739
asymmetrique_profile_score     0.224824
dtype: float64

In [26]:
df_train_full[categorical].corrwith(df_train_full.converted)

Series([], dtype: float64)

In [27]:
train_dict = df_train_full[categorical + numerical].to_dict(orient='records')

In [28]:
from sklearn.feature_extraction import DictVectorizer
 
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [29]:
X_train = dv.transform(train_dict)

In [30]:
X_train[0]

array([  1.,   0.,  nan,   0.,   0.,   0.,  nan,  nan,   0.,   0.,   0.,
        nan,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   1.,   0.,   1.,   0.,   1.,   0.,   0.,   0.,   0.,
         0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,
         0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   1.,   0.,  nan,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   1.,   1.,   0.,   4.,   

In [31]:
dv.get_feature_names()

C:\Users\Lily\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['a_free_copy_of_mastering_the_interview=no',
 'a_free_copy_of_mastering_the_interview=yes',
 'asymmetrique_activity_index',
 'asymmetrique_activity_index=01.high',
 'asymmetrique_activity_index=02.medium',
 'asymmetrique_activity_index=03.low',
 'asymmetrique_activity_score',
 'asymmetrique_profile_index',
 'asymmetrique_profile_index=01.high',
 'asymmetrique_profile_index=02.medium',
 'asymmetrique_profile_index=03.low',
 'asymmetrique_profile_score',
 'city',
 'city=mumbai',
 'city=other_cities',
 'city=other_cities_of_maharashtra',
 'city=other_metro_cities',
 'city=select',
 'city=thane_&_outskirts',
 'city=tier_ii_cities',
 'country',
 'country=asia/pacific_region',
 'country=australia',
 'country=bahrain',
 'country=bangladesh',
 'country=belgium',
 'country=canada',
 'country=china',
 'country=denmark',
 'country=france',
 'country=germany',
 'country=ghana',
 'country=hong_kong',
 'country=india',
 'country=indonesia',
 'country=italy',
 'country=kenya',
 'country=kuwait',
 'c

In [32]:
import math
 
def sigmoid(score):
    return 1 / (1 + math.exp(-score))

In [33]:
def logistic_regression(xi):
    score = bias
    for j in range(n):
        score = score + xi[j] * w[j]
    prob = sigmoid(score)
    return prob

In [34]:
df.drop(['do_not_call',
'through_recommendations',
'what_matters_most_to_you_in_choosing_a_course',
'digital_advertisement',
'newspaper_article',
'search',
'receive_more_updates_about_our_courses',
'update_me_on_supply_chain_content',
'magazine',
'i_agree_to_pay_the_amount_through_cheque'],axis=1,inplace=True

)

In [35]:
df[numerical].corrwith(df.converted)

totalvisits                    0.030395
total_time_spent_on_website    0.362483
page_views_per_visit          -0.003328
asymmetrique_activity_score    0.167962
asymmetrique_profile_score     0.218571
dtype: float64

In [36]:
new_categorical =['lead_origin','lead_source','do_not_email'
             ,'last_activity','country','specialization', 'how_did_you_hear_about_x_education',
             'what_is_your_current_occupation',
            'tags', 'lead_quality','lead_profile','city',
            'asymmetrique_activity_index','asymmetrique_profile_index',
            'a_free_copy_of_mastering_the_interview','last_notable_activity']

In [37]:
df_train_full, df_test= train_test_split(df,test_size=0.2,random_state=1)

df_train, df_val= train_test_split(df_train_full,test_size=0.33, random_state=11)

y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [38]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [39]:
train_dic = df_train[new_categorical+ numerical].to_dict(orient='records')
train_dic

[{'lead_origin': 'lead_add_form',
  'lead_source': 'reference',
  'do_not_email': 'no',
  'last_activity': 'email_opened',
  'country': nan,
  'specialization': 'media_and_advertising',
  'how_did_you_hear_about_x_education': 'select',
  'what_is_your_current_occupation': 'unemployed',
  'tags': 'want_to_take_admission_but_has_financial_problems',
  'lead_quality': 'low_in_relevance',
  'lead_profile': 'potential_lead',
  'city': 'mumbai',
  'asymmetrique_activity_index': nan,
  'asymmetrique_profile_index': nan,
  'a_free_copy_of_mastering_the_interview': 'no',
  'last_notable_activity': 'email_opened',
  'totalvisits': 0.0,
  'total_time_spent_on_website': 0,
  'page_views_per_visit': 0.0,
  'asymmetrique_activity_score': nan,
  'asymmetrique_profile_score': nan},
 {'lead_origin': 'landing_page_submission',
  'lead_source': 'direct_traffic',
  'do_not_email': 'yes',
  'last_activity': 'sms_sent',
  'country': 'india',
  'specialization': 'it_projects_management',
  'how_did_you_hear_

In [40]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dic)

DictVectorizer(sparse=False)

In [41]:
X_train = dv.transform(train_dic)

In [42]:
X_train[0]

array([ 1.,  0., nan,  0.,  0.,  0., nan, nan,  0.,  0.,  0., nan,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0., nan,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0

In [47]:
from sklearn.linear_model import LogisticRegression

In [48]:
model = LogisticRegression(solver='liblinear',random_state=1)
model.fit(X_train,y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').